In [1]:
import os
os.chdir("..")
assert 'PromptDataExtract' in os.getcwd()

In [2]:
import json
from backend.utils import Frame
from pymongo import MongoClient
from backend import sett

In [3]:
prop_meta = json.load(open('data/property_metadata.json'))

In [4]:
def connect():
    client = MongoClient(
        host='localhost',
        port=8161,
    )
    db = client['corpus']
    return db

mongo = connect()
collection = mongo['modular_run_4_PS']

In [8]:
df = Frame()
propcount = Frame()

In [9]:
# Loop over all known properties
for name, meta in prop_meta.items():
    othernames = meta['property_list']

    records = list(collection.find({
        "material_records.property_record.entity_name": {'$in': othernames }
    }))

    propcount.add(property=name, count=len(records))

    for record in records:
        for mat in record['material_records']:
            prop = mat['property_record']

            if prop['entity_name'] in othernames:
                df.add(doi=record['DOI'], prop=name, value=prop['property_numeric_value'])

propcount.df

,property,count
0,gravimetric energy density,1427
1,gravimetric power density,2188
2,specific capacitance,2837
3,current density,1082
4,areal current density,1082
...,...,...
77,luminance,714
78,luminance efficiency,388
79,dielectric loss,330
80,capacitance retention,343


In [10]:
df.save('notebooks/Abstract-Extracted_Data.csv')
propcount.save('notebooks/Abstract-Property_Count.csv')

In [11]:
propcount.df.sum()

property    gravimetric energy densitygravimetric power de...
count                                                   95781
dtype: object